In [5]:
import pandas as pd

import re
from nltk.stem import PorterStemmer


In [7]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification


c:\Users\utsag\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load the datasets
real_news = pd.read_csv('True.csv')

# Printing head and tail
print(real_news)

# Print length => rows
print("Length (rows):", len(real_news))

                                                   title  \
0      As U.S. budget fight looms, Republicans flip t...   
1      U.S. military to accept transgender recruits o...   
2      Senior U.S. Republican senator: 'Let Mr. Muell...   
3      FBI Russia probe helped by Australian diplomat...   
4      Trump wants Postal Service to charge 'much mor...   
...                                                  ...   
21412  'Fully committed' NATO backs new U.S. approach...   
21413  LexisNexis withdrew two products from Chinese ...   
21414  Minsk cultural hub becomes haven from authorities   
21415  Vatican upbeat on possibility of Pope Francis ...   
21416  Indonesia to buy $1.14 billion worth of Russia...   

                                                    text       subject  \
0      WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1      WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2      WASHINGTON (Reuters) - The special counsel inv... 

In [ ]:
# Load the datasets
fake_news = pd.read_csv('Fake.csv')

# Printing head and tail
print(fake_news)

# Print length => rows
print("Length (rows):", len(fake_news))

                                                   title  \
0       Donald Trump Sends Out Embarrassing New Year’...   
1       Drunk Bragging Trump Staffer Started Russian ...   
2       Sheriff David Clarke Becomes An Internet Joke...   
3       Trump Is So Obsessed He Even Has Obama’s Name...   
4       Pope Francis Just Called Out Donald Trump Dur...   
...                                                  ...   
23476  McPain: John McCain Furious That Iran Treated ...   
23477  JUSTICE? Yahoo Settles E-mail Privacy Class-ac...   
23478  Sunnistan: US and Allied ‘Safe Zone’ Plan to T...   
23479  How to Blow $700 Million: Al Jazeera America F...   
23480  10 U.S. Navy Sailors Held by Iranian Military ...   

                                                    text      subject  \
0      Donald Trump just couldn t wish all Americans ...         News   
1      House Intelligence Committee Chairman Devin Nu...         News   
2      On Friday, it was revealed that former Milwauk...    

In [4]:
# Load the dataset
fake_news = pd.read_csv('Fake.csv')
real_news = pd.read_csv('True.csv')

# Combine the datasets and shuffle
fake_news['label'] = 0    # adds a column
real_news['label'] = 1

data = pd.concat([fake_news, real_news]).sample(frac=1.0)   # frac 1.0 => returns all the data (rows)

# Write the combined data to a new CSV file
data.to_csv('Combined.csv', index=False)

print("Data loaded successfully and combined CSV generated!")

data

Data loaded successfully and combined CSV generated!


,title,text,subject,date,label
7280,Rachel Maddow Just Exposed The Worst Case Of ...,The game plan for Republicans isn t exactly di...,News,"March 26, 2016",0
2588,"Medicaid, pension costs create budget complica...",NEW YORK (Reuters) - A sluggish forecast for U...,politicsNews,"July 24, 2017",1
8335,Senator Grassley could be persuaded to hold he...,WASHINGTON (Reuters) - U.S. Senator Chuck Gras...,politicsNews,"August 30, 2016",1
9443,AWESOME! Conservative Artist Crashes Anti-Trum...,Our favorite conservative street artist Sabo c...,politics,"Nov 13, 2017",0
12664,Did OPRAH Just Leave “Nasty” Hillary Wishing S...,"Four months ago, Oprah, the woman who made her...",politics,"Oct 22, 2016",0
...,...,...,...,...,...
2300,Trump administration goes on attack against le...,WASHINGTON (Reuters) - U.S. Attorney General J...,politicsNews,"August 4, 2017",1
564,Trump Crushed As Ex-KKK Leader Bashes Him For...,Donald Trump is in big trouble. Not only has h...,News,"August 14, 2017",0
7014,"John McCain Brushes Off Hispanic Voters, Says...",When you re running for re-election in your to...,News,"April 9, 2016",0
17154,"Russian, Israeli leaders discuss Iran nuclear ...",MOSCOW (Reuters) - Russian President Vladimir ...,worldnews,"October 18, 2017",1


In [6]:
stemmer = PorterStemmer()

def preprocess_text(text):
    # Replace characters that are not between a to z or A to Z with whitespace
    text = re.sub('[^a-zA-Z]', ' ', text)

    # Convert all characters into lowercase
    text = text.lower()

    # Remove inflectional morphemes like "ed", "est", "s", and "ing" from their token stem
    text = [stemmer.stem(word) for word in text.split()]

    # Join the processed words back into a single string
    text = ' '.join(text)

    return text

In [12]:
# Check if GPU is available
if tf.test.is_gpu_available():
    device_name = tf.test.gpu_device_name()
else:
    device_name = 'CPU:0'
print('Using device:', device_name)

# Load the dataset
data = pd.read_csv('Combined.csv')

# Preprocess the text using the preprocessing function
print("\nPreprocessing data...")

data['title_preprocessed'] = data['title'].apply(preprocess_text)

# Split the dataset into train, validation, and test sets
train_ratio = 0.64
val_ratio = 0.16
test_ratio = 0.2

print("\nSplitting Data...")

train_data = data.sample(frac=train_ratio, random_state=42)
remaining_data = data.drop(train_data.index)

val_data = remaining_data.sample(frac=val_ratio/(val_ratio+test_ratio), random_state=42)
test_data = remaining_data.drop(val_data.index)

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the titles and convert them into BERT input tensors
print("\nTokenizing and Converting...")

train_inputs = tokenizer(list(train_data['title_preprocessed']), truncation=True, padding='max_length', max_length=42, return_tensors='tf')
val_inputs = tokenizer(list(val_data['title_preprocessed']), truncation=True, padding='max_length', max_length=42, return_tensors='tf')
test_inputs = tokenizer(list(test_data['title_preprocessed']), truncation=True, padding='max_length', max_length=42, return_tensors='tf')

# Convert the labels into TensorFlow tensors
train_labels = tf.convert_to_tensor(list(train_data['label']))
val_labels = tf.convert_to_tensor(list(val_data['label']))
test_labels = tf.convert_to_tensor(list(test_data['label']))

# Extract token tensors, segment tensors, and mask tensors from the BERT inputs
train_token_tensors = train_inputs['input_ids']
train_segment_tensors = train_inputs['token_type_ids']
train_mask_tensors = train_inputs['attention_mask']

val_token_tensors = val_inputs['input_ids']
val_segment_tensors = val_inputs['token_type_ids']
val_mask_tensors = val_inputs['attention_mask']

test_token_tensors = test_inputs['input_ids']
test_segment_tensors = test_inputs['token_type_ids']
test_mask_tensors = test_inputs['attention_mask']

# Build the BERT model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Compile the model
print("\nCompiling Model...")

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Train the BERT model
print("\nTraining Model...")

batch_size = 128
num_epochs = 1

history = model.fit([train_token_tensors, train_segment_tensors, train_mask_tensors], train_labels, batch_size=batch_size, epochs=num_epochs, validation_data=([val_token_tensors, val_segment_tensors, val_mask_tensors], val_labels))

# Save the trained model
print("\nSaving Model...")

model.save_pretrained('/content/bertv3_model')


Using device: CPU:0

Preprocessing data...

Splitting Data...

Tokenizing and Converting...


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Compiling Model...

Training Model...
  2/225 [..............................] - ETA: 1:01:18 - loss: 0.7370 - accuracy: 0.4844

KeyboardInterrupt: 

In [9]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate([test_token_tensors, test_segment_tensors, test_mask_tensors], test_labels, batch_size=batch_size)
print(f'Test loss: {loss * 100:.3f}%')
print(f'Test accuracy: {accuracy * 100:.3f}%')


141/141 [==============================] - 374s 3s/step - loss: 0.1216 - accuracy: 0.9550
Test loss: 12.158%
Test accuracy: 95.501%


In [10]:
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

# Load the saved model
model = TFBertForSequenceClassification.from_pretrained('/content/bertv3_model')

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example input text
input_text = input("\n\nEnter News Title: ")

# Preprocess the input text
preprocessed_text = preprocess_text(input_text)

print("Preprocessed Text:", preprocessed_text)

# Tokenize the preprocessed text
inputs = tokenizer(preprocessed_text, truncation=True, padding='max_length', max_length=42, return_tensors='tf')

# Extract input tensors
token_tensors = inputs['input_ids']
segment_tensors = inputs['token_type_ids']
mask_tensors = inputs['attention_mask']

# Make predictions
predictions = model.predict([token_tensors, segment_tensors, mask_tensors])
logits = predictions.logits[0]
probabilities = tf.nn.softmax(logits)
predicted_label = tf.argmax(probabilities)

# Print the predicted label and probabilities
if predicted_label == 0:
    print("\n*-*-Fake News-*-*")
else:
    print("\n*-*-Real News-*-*")

print("\nProbability of being fake: {:.2%}".format(probabilities[0]))
print("Probability of being real: {:.2%}".format(probabilities[1]))


OSError: Incorrect path_or_model_id: '/content/bertv3_model'. Please provide either the path to a local folder or the repo_id of a model on the Hub.